# DSTI ML Project A22 Cohort: Book Rating Predictor
The aim of the project is to train and evaluate different models that predict a book’s rating from a GoodReads dataset

In [1]:
# Ensure to select the .venv-book kernel before importing
# Kernel > Change kernel > .venv-book

import datetime
import pandas as pd
from sklearn import preprocessing

## 2) Feature Engineering

### 2.1) Preprocessing of dataset
As seen previously, we will drop the ```isbn``` column.

For the ```title```,  ```authors``` and  ```publisher``` variables we will use **Label Encoding** as they have a large number of "categories". We can note also that the ```title``` and  ```authors``` columns will not have much impact on the target variable due to the very large number of "categories" and hence may not be useful to train the model.

For the ```publication_date``` we will convert the date into a continuous variable by:
- i) Subtracting all the dates with a reference date
- ii) Taking only the year

For the ```language_code```, since we have only two dummies, we will convert the original column as a boolean variable instead of doing One-Hot Encoding.

| Variable | Pre-processing | New Features |
| --- | --- | --- |
| ```average_rating``` | None |  |
| ```bookID``` | None |  |
| ```isbn13``` | None |  |
| ```isbn``` | Drop |  |
| ```ratings_count```| Quantile Regrouping | ```ratings_count_p_10```|
| ```text_reviews_count``` | Quantile Regrouping | ```text_reviews_count_p_10``` |
| ```num_pages``` | Quantile Regrouping | ```text_reviews_count_p_10``` |
| ```publication_date``` | Date Transformation | ```publication_year``` |
| ```language_code``` | Standardization (Boolean) |  |
| ```publisher``` | Label Encoding |  |
| ```title``` | Label Encoding | ```title_len``` |
| ```authors``` | Split authors & Label Encoding | ```author1``` + ```author2``` + ```num_authors``` |


In [2]:
# Read the original dataset
df = pd.read_csv("../data/books.csv", on_bad_lines='skip')
# Clean the columns name
for col in df.columns:
    df = df.rename(columns = {col: col.strip()})
print(f"Original shape of dataset: {df.shape}")

Original shape of dataset: (11123, 12)


#### 2.1.1) Dropping ```isbn``` column

In [3]:
df_isbn_drop = df.drop(["isbn"], axis=1)
df_isbn_drop.shape

(11123, 11)

#### 2.1.2) Quantile Regrouping

In [4]:
df_Q = df_isbn_drop.copy()
vars = ["num_pages", "ratings_count", "text_reviews_count"]
p = 10
for var in vars:
    var_Q = pd.qcut(df_Q[var], p, labels=False)
    var_Q = var_Q.rename(f"{var}_p_{p}")
    df_Q = pd.concat([df_Q, var_Q], axis=1)
df_Q.shape

(11123, 14)

#### 2.1.3) Date Transformation of ```publication_date```

In [5]:
# Check for valid dates in the dataset
df_date_trans = df_Q.copy()
invalid_date = []
for index, row in df_date_trans.iterrows():
    p_date = df_date_trans.loc[index, "publication_date"]
    try:
        p_date2 = datetime.datetime.strptime(p_date,'%m/%d/%Y').date()
    except Exception as e:
        invalid_date.append([index, p_date, e])

pd.DataFrame(invalid_date, columns=["index", "date", "error"])

,index,date,error
0,8177,11/31/2000,day is out of range for month
1,11094,6/31/1982,day is out of range for month


In [6]:
print(f"Before: {len(df_date_trans)}")
for id in invalid_date:
    df_date_trans = df_date_trans.drop([id[0]])
print(f"After: {len(df_date_trans)}")

Before: 11123
After: 11121


We convert the date into a continuous value by:
- i) Subtracting all date with a reference date
- ii) Taking only the year

In [7]:
REF_DATE = datetime.date(2023, 8, 4)
for index, row in df_date_trans.iterrows():
    p_date = df_date_trans.loc[index, "publication_date"]
    p_date2 = datetime.datetime.strptime(p_date,'%m/%d/%Y').date()
    num_days = (REF_DATE - p_date2).days
    df_date_trans.at[index, "publication_date"] = num_days
    df_date_trans.at[index, "publication_year"] = p_date2.year

# Ensure the dates are of numeric type
df_date_trans["publication_date"] = df_date_trans["publication_date"].astype(int)
df_date_trans["publication_year"] = df_date_trans["publication_year"].astype(int)
df_date_trans.shape

(11121, 15)

#### 2.1.4) Standardization of the ```language_code``` 
We regroup values of ```language_code``` into only english (1) and others (0)

In [8]:
df_2_dummies = df_date_trans.copy()

for index, row in df_2_dummies.iterrows():
    if "en" in row["language_code"]:
        df_2_dummies.loc[index, "language_code"] =  1
    else:
        df_2_dummies.loc[index, "language_code"] = 0

df_2_dummies.shape

(11121, 15)

#### 2.1.5) Make ```title_len```

In [9]:
df_len_title = df_2_dummies.copy()
for index, row in df_len_title.iterrows():
    df_len_title.loc[index, "title_len"] = len(row["title"])

df_len_title["title_len"] = df_len_title["title_len"].astype(int)
df_len_title.shape

(11121, 16)

#### 2.1.6) Split ```authors``` and make ```num_authors```

In [10]:
df_authors_split = df_len_title.copy()
for index, row in df_authors_split.iterrows():
    authors = row["authors"].split("/")
    df_authors_split.loc[index, "num_authors"] = len(authors)
    for i, author in enumerate(authors[0:2]):
        df_authors_split.loc[index, f"author_{i+1}"] = author

df_authors_split["num_authors"] = df_authors_split["num_authors"].astype(int)
df_authors_split = df_authors_split.drop(columns="authors")
df_authors_split.shape

(11121, 18)

#### 2.1.7) Label Encoding 
We apply **Label Encoding** to the 3 columns with text "categories":
- ```title```
- ```authors```
- ```author_1```
- ```author_2```
- ```publisher```



In [11]:
df_label_encoded = df_authors_split.copy()
labelEncode = preprocessing.LabelEncoder()
for var in ["title", "author_1", "author_2", "publisher"]:
    df_label_encoded[var] = labelEncode.fit_transform(df_label_encoded[var])

print(df_label_encoded.shape)
df_label_encoded.head(10)

(11121, 18)


,bookID,title,average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,num_pages_p_10,ratings_count_p_10,text_reviews_count_p_10,publication_year,title_len,num_authors,author_1,author_2
0,1,3070,4.57,9780439785969,1,652,2095690,27591,6166,1777,9,9,9,2006,57,2,1598,2030
1,2,3071,4.49,9780439358071,1,870,2153167,29221,6911,1777,9,9,9,2004,60,2,1598,2030
2,4,3068,4.42,9780439554893,1,352,6333,244,7216,1772,6,7,7,2003,58,1,1598,3236
3,5,3073,4.56,9780439655484,1,435,2339585,36325,7034,1777,7,9,9,2004,59,2,1598,2030
4,8,3063,4.78,9780439682589,1,2690,41428,164,6899,1772,9,9,6,2004,54,2,1598,2030
5,9,9689,3.74,9780976540601,1,152,19,1,6674,1366,1,0,0,2005,87,1,4064,3236
6,10,3064,4.73,9780439827607,1,3342,28242,808,6535,1772,9,9,8,2005,44,1,1598,3236
7,12,9117,4.38,9780517226957,1,815,3628,254,6485,788,9,7,7,2005,108,1,985,3236
8,13,9116,4.38,9780345453747,1,815,249558,4080,7766,524,9,9,9,2002,86,1,985,3236
9,14,7796,4.22,9781400052929,1,215,4930,460,6940,470,2,7,8,2004,75,1,985,3236


So we finally ended up to 18 columns purely numeric from 12 columns initially, we will now need to make a feature selection by looking which features got more impact on our target (```average_rating```)

#### 2.1.8) Re-arrange columns & save pre-processed dataset

In [12]:
# Re-arrange and save the pre-processed dataset
rearranged_vars = [
    "average_rating",
    "bookID",
    "isbn13",
    "num_pages",
    'num_pages_p_10', 
    "ratings_count",
    'ratings_count_p_10', 
    "text_reviews_count",
    'text_reviews_count_p_10',
    "publication_date",
    "publication_year", 
    "language_code",
    "title",
    'title_len',
    "author_1", 
    "author_2",
    'num_authors',
    "publisher", 
]

df_final = df_label_encoded[rearranged_vars]
df_final.to_csv("../data/books_preprocessed.csv", index=False)
df_final.shape

(11121, 18)

### 2.2) Feature Analysis

#### 2.2.1) Correlation b/w target and features from non-numeric variables

In [13]:
vars = [
    "average_rating", 
    "language_code",
    "title",
    'title_len',
    "author_1", 
    "author_2",
    'num_authors',
    "publisher", 
]
corr = abs(df_final[vars].corr())
corr.style.background_gradient(cmap='coolwarm')

,average_rating,language_code,title,title_len,author_1,author_2,num_authors,publisher
average_rating,1.000000,0.033717,0.026518,0.096451,0.021646,0.051174,0.018947,0.027408
language_code,0.033717,1.000000,0.052061,0.069927,0.004082,0.021827,0.009668,0.002911
title,0.026518,0.052061,1.000000,0.032312,0.029074,0.000708,0.002161,0.024765
title_len,0.096451,0.069927,0.032312,1.000000,0.027891,0.022055,0.036263,0.007670
author_1,0.021646,0.004082,0.029074,0.027891,1.000000,0.016950,0.006355,0.036655
author_2,0.051174,0.021827,0.000708,0.022055,0.016950,1.000000,0.385501,0.001582
num_authors,0.018947,0.009668,0.002161,0.036263,0.006355,0.385501,1.000000,0.010010
publisher,0.027408,0.002911,0.024765,0.007670,0.036655,0.001582,0.010010,1.000000


In [14]:
# Absolute correlation b/w target and features
corr2 = pd.DataFrame(corr.iloc[1:-1, 0])
corr2.style.background_gradient(cmap='coolwarm')

,average_rating
language_code,0.033717
title,0.026518
title_len,0.096451
author_1,0.021646
author_2,0.051174
num_authors,0.018947


We see very low correlation b/w the features and the target, however we found a slightly higher correlation b/w the target and our "engineered" features like ```title_len``` compared to ```title```. However the engineered feature ```num_authors``` has the lowest correlation.

#### 2.2.2) Correlation b/w target and features from numeric variables

In [15]:
vars = [
    "average_rating",
    "bookID",
    "isbn13",
    "num_pages",
    'num_pages_p_10', 
    "ratings_count",
    'ratings_count_p_10', 
    "text_reviews_count",
    'text_reviews_count_p_10',
    "publication_date",
    "publication_year", 
]
corr = abs(df_final[vars].corr())
corr.style.background_gradient(cmap='coolwarm')

,average_rating,bookID,isbn13,num_pages,num_pages_p_10,ratings_count,ratings_count_p_10,text_reviews_count,text_reviews_count_p_10,publication_date,publication_year
average_rating,1.000000,0.036259,0.001967,0.150430,0.103904,0.038228,0.115798,0.033669,0.069308,0.030897,0.031730
bookID,0.036259,1.000000,0.002197,0.026463,0.009094,0.076803,0.097595,0.081262,0.107525,0.027796,0.027861
isbn13,0.001967,0.002197,1.000000,0.009832,0.006097,0.005495,0.032460,0.008153,0.032120,0.000665,0.000335
num_pages,0.150430,0.026463,0.009832,1.000000,0.813550,0.034371,0.122664,0.037021,0.103089,0.018278,0.019259
num_pages_p_10,0.103904,0.009094,0.006097,0.813550,1.000000,0.049203,0.179611,0.058205,0.164273,0.003962,0.003143
ratings_count,0.038228,0.076803,0.005495,0.034371,0.049203,1.000000,0.236985,0.865977,0.232485,0.043924,0.044542
ratings_count_p_10,0.115798,0.097595,0.032460,0.122664,0.179611,0.236985,1.000000,0.296684,0.950245,0.125350,0.125324
text_reviews_count,0.033669,0.081262,0.008153,0.037021,0.058205,0.865977,0.296684,1.000000,0.299551,0.065953,0.066883
text_reviews_count_p_10,0.069308,0.107525,0.032120,0.103089,0.164273,0.232485,0.950245,0.299551,1.000000,0.146236,0.146503
publication_date,0.030897,0.027796,0.000665,0.018278,0.003962,0.043924,0.125350,0.065953,0.146236,1.000000,0.999396


In [16]:
# Absolute correlation b/w target and features
corr2 = pd.DataFrame(corr.iloc[1:-1, 0])
corr2.style.background_gradient(cmap='coolwarm')

,average_rating
bookID,0.036259
isbn13,0.001967
num_pages,0.150430
num_pages_p_10,0.103904
ratings_count,0.038228
ratings_count_p_10,0.115798
text_reviews_count,0.033669
text_reviews_count_p_10,0.069308
publication_date,0.030897


We have slightly higher correlations for numeric features and we can see that our Quantile Regrouping on some features was efficient.
For the ```ratings_count``` and ```text_reviews_count```, we get higher correlations with their corresponding "engineered" features which will probably help our model to perform better, but not for the ```num_pages``` column.

We can see also that the ```publication_year``` has a slightly higher correlation with the target than the ```publication_date```. But by keeping only the year, we loose some information which maybe useful to train the model, hence we cannot really know which one to keep and we will find out experimentally in the next section.

For the unique identifiers, surprisingly, the ```bookID``` seems to have some correlation to the target, however it does not make sense to keep the ```isbn13``` column due to its very low correlation with the target.

**Synthesis**:
- We will drop both the ```isbn13``` for the above reasons.
- However, due to the generally low correlation of the features with the target, we cannot know which features to select, as for some of the "engineered" features, we have gain in correlation but we have lost some information, hence we do not know how the models will interpret these features. Therefore, we will find **experimentally** the best set of features in the next section by evaluatin with the models.